# 1.模型部署简介

模型部署： 算法模型 -->  应用程序

希望通过本教程，带领大家学会如何把自己的 PyTorch 模型部署到 ONNX Runtime/TensorRT 上。

* 中间表示 ONNX 的定义标准
* PyTorch 模型转换到 ONNX 模型的方法
* 推理引擎 ONNX Runtime、TensorRT 的使用方法
* 部署流水线 PyTorch - ONNX - ONNX Runtime/TensorRT 的示例及常见部署问题的解决方法
* MMDeploy C/C++ 推理 SDK


软件工程领域的“部署”：把开发完毕的软件投入使用的过程，包括环境配置、软件安装等。对于深度学习模型而言，模型部署指让训练好的模型在特定的环境中运行的过程。这过程中需要解决：环境适配、运行性能等问题。

部署流水线：

1. 训练：使用深度学习框架训练模型的参数，训练框架包括了：Pytorch、TensorFlow、Caffe、MXNet等
2. 中间表示（优化）：将不同框架下训练的模型结构定义和权重参数，转换为中间表达，常见的中间表达有：ONNX、TorchScript、Caffe等
3. 推理引擎（运行）：将中间表示转化为特定的优化的推理引擎的专用格式。常见的推理引擎有：TensorRT、ONNXRuntime、PPL、NCNN、OpenVINO等。

# 2.部署第一个模型

Pytorch Module -> ONNX -> ONNX Runtime

## 2.1 定义一个简单的图像超分的模型

该超分模型由一个上采样层和3层卷积层构成。

In [1]:
import os
import cv2
import numpy as np

import requests
import torch
import torch.onnx
from torch import nn


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor):
        super().__init__()
        self.upsampler = nn.Upsample(
            scale_factor=upscale_factor, mode="bicubic", align_corners=False
        )
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)  # 压缩channel
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.upsampler(x)
        out = self.relu(self.conv1(out))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out

In [2]:
model = SuperResolutionNet(2.0)
x = torch.randn((1, 3, 224, 224))
for modules in model.named_modules():
    if modules[0] == "" or modules[0] == "relu":
        continue
    shape = x.shape
    x = model.get_submodule(modules[0])(x)
    print(f"{modules[0]}: {list(shape)} => {list(x.shape)}")

upsampler: [1, 3, 224, 224] => [1, 3, 448, 448]
conv1: [1, 3, 448, 448] => [1, 64, 448, 448]
conv2: [1, 64, 448, 448] => [1, 32, 448, 448]
conv3: [1, 32, 448, 448] => [1, 3, 448, 448]


## 2.2 加载训练好的模型权重

In [3]:
urls = [
    "https://download.openmmlab.com/mmediting/restorers/srcnn/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth",
    "https://raw.githubusercontent.com/open-mmlab/mmediting/master/tests/data/face/000001.png",
]

names = ["srcnn.pth", "face.png"]

for url, name in zip(urls, names):
    if not os.path.exists(name):
        open(name, "wb").write(requests.get(url).content)

# 加载离线权重，并对key进行适配
state_dicts = torch.load("srcnn.pth")["state_dict"]
print(state_dicts.keys())
for old_key in list(state_dicts.keys()):
    new_key = old_key.replace("generator.", "", 1)
    state_dicts[new_key] = state_dicts.pop(old_key)
print(state_dicts.keys())
srnn_model = SuperResolutionNet(upscale_factor=3)
srnn_model.load_state_dict(state_dict=state_dicts)

odict_keys(['generator.conv1.weight', 'generator.conv1.bias', 'generator.conv2.weight', 'generator.conv2.bias', 'generator.conv3.weight', 'generator.conv3.bias'])
odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias'])


<All keys matched successfully>

## 2.3 使用Pytorch进行推理

In [4]:
input_image = cv2.imread("./face.png").astype(np.float32)
input_image = np.transpose(input_image, (2, 0, 1))  # H,W,C -> C,H,W
input_image = np.expand_dims(input_image, 0)  # C,H,W -> 1,C,H,W

torch_output = srnn_model(torch.from_numpy(input_image)).detach().numpy()
torch_output = np.squeeze(torch_output, 0)
torch_output = np.clip(torch_output, 0, 255)
torch_output = np.transpose(torch_output, (1, 2, 0)).astype(np.uint8)

cv2.imwrite("./outputs/face_torch_1.png", torch_output)

True

## 2.4 导出为ONNX格式的模型

In [5]:
import onnx

x = torch.randn(1, 3, 256, 256)
with torch.no_grad():
    torch.onnx.export(
        model=srnn_model,
        args=x,
        f="./outputs/srcnn1.onnx",
        opset_version=11,
        input_names=["input"],
        output_names=["output"],
    )

# 使用onnx来加载序列化的模型，加载出来的就是一个ModelProto对象
onnx_model = onnx.load("./outputs/srcnn1.onnx")
# 检查ModelProto是否正常
onnx.checker.check_model(onnx_model, full_check=True)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## 2.5 使用Netron来查看Onnx文件

地址：[netron.app](https://netron.app/)

<div align="center">
  <img src="./assets/srcnn.onnx.svg" height="600"/> </div>

## 2.6 使用OnnxRuntime进行推理

In [6]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("./outputs/srcnn1.onnx")
ort_inputs = {"input": input_image}

# 输入输出都是numpy.ndarry
ort_output = ort_session.run(output_names=["output"], input_feed=ort_inputs)[0]

ort_output = np.squeeze(ort_output, 0)
ort_output = np.clip(ort_output, 0, 255)
ort_output = np.transpose(ort_output, (1, 2, 0)).astype(np.uint8)

successed = cv2.imwrite("./outputs/face_ort_1.png", ort_output)

# 3. 解决模型部署中的难题

上面部署的模型，十分顺利，但是它有一个缺陷，图片的放大倍数被写死在模型结构中了，我们在推理时，无法动态的调整。

另外上面模型部署的简单也主要是因为SCRNN结构比较简单，只包括了：`Upsample`、`Conv2d`、`Relu`这3个标准的算子。这些算法在各个中间表示和推理引擎上已经得到完美的支持。如果模型的操作稍微复杂一点，我们可能就要为兼容模型而付出大量的功夫了。在实际部署时，我们可能会遇到几下几类的困难：

* 模型的动态化。出于性能的考虑，各推理框架都默认模型的输入形状、输出形状、结构是静态的。而为了让模型的泛化性更强，部署时需要在尽可能不影响原有逻辑的前提下，让模型的输入输出或是结构动态化。
* 新算子的实现。深度学习技术日新月异，提出新算子的速度往往快于 ONNX 维护者支持的速度。为了部署最新的模型，部署工程师往往需要自己在 ONNX 和推理引擎中支持新算子。
* 中间表示与推理引擎的兼容问题。由于各推理引擎的实现不同，对 ONNX 难以形成统一的支持。为了确保模型在不同的推理引擎中有同样的运行效果，部署工程师往往得为某个推理引擎定制模型代码，这为模型部署引入了许多工作量。

## 3.1 SRCNN支持动态设置放大倍数

下面我们对前面的`SRCNN`网络结构进行修改，将上采样写在`forward`阶段，调用`interpolate`函数来实现。

In [7]:
import torch.nn.functional as F


class SuperResolutionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)  # 压缩channel
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

        self.relu = nn.ReLU()

    def forward(self, x, upscale_factor):
        out = F.interpolate(
            x, scale_factor=upscale_factor, mode="bicubic", align_corners=False
        )
        out = self.relu(self.conv1(out))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out


torch_model = SuperResolutionNet()
torch_model.load_state_dict(state_dicts)
torch_model.eval()

SuperResolutionNet(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU()
)

In [8]:
upscale_factor = 3.0
torch_output = (
    torch_model(torch.from_numpy(input_image), upscale_factor).detach().numpy()
)
torch_output = np.squeeze(torch_output, 0)
torch_output = np.clip(torch_output, 0, 255)
torch_output = np.transpose(torch_output, (1, 2, 0)).astype(np.uint8)
cv2.imwrite("./outputs/face_torch_2.png", torch_output)

True

由于pytorch本身具有动态性，所以上面的过程没有任何问题，我们在forward时，指定了放大倍数。现在我们来尝试将上面的模型转换为ONNX模型。

In [9]:
with torch.no_grad():
    torch.onnx.export(
        model=torch_model,
        args=(x, 3),
        f="./outputs/srcnn2.onnx",
        opset_version=11,
        input_names=["input", "factor"],
        output_names=["output"],
    )

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



TypeError: upsample_bicubic2d() received an invalid combination of arguments - got (Tensor, NoneType, bool, list), but expected one of:
 * (Tensor input, tuple of ints output_size, bool align_corners, tuple of floats scale_factors)
      didn't match because some of the arguments have invalid types: (Tensor, !NoneType!, bool, !list of [Tensor, Tensor]!)
 * (Tensor input, tuple of ints output_size, bool align_corners, float scales_h, float scales_w, *, Tensor out)


上面报错的原因是，在导出到ONXX模型时，模型的输入参数的类型必须全部是torch.Tensor。而我们传入的第二个参数“3”是一个整形变量。这不符合Pytorch转ONNX的规定。现在我们可以尝试修改一下模型：

In [10]:
class SuperResolutionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)  # 压缩channel
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

        self.relu = nn.ReLU()

    def forward(self, x, upscale_factor):
        out = F.interpolate(
            x, scale_factor=upscale_factor.item(), mode="bicubic", align_corners=False
        )
        out = self.relu(self.conv1(out))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out


torch_model = SuperResolutionNet()
torch_model.load_state_dict(state_dicts)
torch_model.eval()

with torch.no_grad():
    torch.onnx.export(
        model=torch_model,
        args=(x, torch.tensor(3)),
        f="./outputs/srcnn2.onnx",
        opset_version=11,
        input_names=["input", "factor"],
        output_names=["output"],
    )

/tmp/ipykernel_3137538/646510743.py:12: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x, scale_factor=upscale_factor.item(), mode="bicubic", align_corners=False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



转换成功了，但报了一个Warning，意思是虽然转成功了，但在转换过程中，发现了把一个Tensor转换为了一个Python number，这会导致导出的ONNX模型，把这个值视为一个常量。也就是说导出的模型，我们还是无法动态的设置upscale_factor。

如果我们在netron.app中加载刚才导出来的ONNX文件，会发现，还是只有一个输入。

## 3.2 自定义ONNX算子解决问题

仔细观察 Netron 上可视化出的 ONNX 模型，可以发现在 PyTorch 中无论是使用最早的 nn.Upsample，还是后来的 interpolate，PyTorch 里的插值操作最后都会转换成 ONNX 定义的 Resize 操作。也就是说，所谓 PyTorch 转 ONNX，实际上就是把每个 PyTorch 的操作映射成了 ONNX 定义的算子。

其中`Resize`算子的输入是：`X`、`roi`、`scales`。`scales`是一个长度为4的一维张量，其内容为`[1,1,3,3]`代表的是各个维度的缩放系数。下面我们来尝试自定义一个pytorch的算子，希望把它映射到ONXX的Resize算子上。

In [11]:
class NewInterpolate(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, scales):
        scales = scales.tolist()[-2:]
        return F.interpolate(
            input, scale_factor=scales, mode="bicubic", align_corners=False
        )

    @staticmethod
    def symbolic(g, input, scales):
        return g.op(
            "Resize",
            input,
            g.op("Constant", value_t=torch.tensor([], dtype=torch.float32)),
            scales,
            coordinate_transformation_mode_s="pytorch_half_pixel",
            cubic_coeff_a_f=-0.75,
            mode_s="cubic",
            nearest_mode_s="floor",
        )

映射到 ONNX 的方法由一个算子的 `symbolic` 方法决定。`symbolic` 方法第一个参数必须是`g`，之后的参数是算子的自定义输入，和 `forward` 函数一样。ONNX 算子的具体定义由 `g.op` 实现。`g.op` 的每个参数都可以映射到 ONNX 中的算子属性：

<div align="center">
  <img src="./assets/symbolic_onnx_op.png" height="400"/> </div>

In [12]:
class StrangeSuperResolutionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)  # 压缩channel
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

        self.relu = nn.ReLU()

    def forward(self, x, upscale_factor):
        out = NewInterpolate.apply(x, upscale_factor)
        out = self.relu(self.conv1(out))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out


torch_model = StrangeSuperResolutionNet()
torch_model.load_state_dict(state_dicts)
torch_model.eval()


upscale_factor = torch.tensor([1, 1, 3, 3], dtype=torch.float32)
torch_output = (
    torch_model(torch.from_numpy(input_image), upscale_factor).detach().numpy()
)
torch_output = np.squeeze(torch_output, 0)
torch_output = np.clip(torch_output, 0, 255)
torch_output = np.transpose(torch_output, (1, 2, 0)).astype(np.uint8)
cv2.imwrite("./outputs/face_torch_3.png", torch_output)

with torch.no_grad():
    torch.onnx.export(
        model=torch_model,
        args=(x, upscale_factor),
        f="./outputs/srcnn3.onnx",
        opset_version=11,
        input_names=["input", "factor"],
        output_names=["output"],
    )

/tmp/ipykernel_3137538/3127156981.py:4: TracerWarning: Converting a tensor to a Python list might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scales = scales.tolist()[-2:]


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



现在我们已经成功导出来了新的ONXX模型，虽然上面依然有一条Tracing的Warning，但我们通过Netron.APP看到导出的ONNX算子，已经支持两个输入了。

<div align="center">
  <img src="./assets/new_onnx_model.png" height="400"/> </div>

In [ ]:
ort_session = onnxruntime.InferenceSession("./outputs/srcnn3.onnx")

ort_inputs = {"input": input_image, "factor": upscale_factor.numpy()}

# 输入输出都是numpy.ndarry
ort_output = ort_session.run(output_names=["output"], input_feed=ort_inputs)[0]

ort_output = np.squeeze(ort_output, 0)
ort_output = np.clip(ort_output, 0, 255)
ort_output = np.transpose(ort_output, (1, 2, 0)).astype(np.uint8)

successed = cv2.imwrite("./outputs/face_ort_3.png", ort_output)

OK，导出的ONNX模型在ONNXRuntime中也能成功运行了；回顾一下，最开始不能直接导出的主要原因是，原始的`interpolate`算子的第二个参数是一个python number，它在向ONNX进行算子转换时，会被当作一个Çonstant进行转换。所以如果能把第二个参数改为一个`Tensor`就可以了，所以我们手动定义了一个WrapInterpolate函数，并通完了`symbolic`来手动定义它与ONNX算子之间的映射关系。